In [1]:
import pandas as pd


In [2]:
df = pd.read_csv("D:/study/Data Mining/Project/input/tweets.csv")

In [3]:
today = pd.to_datetime("today")
print(today)

now = pd.to_datetime("now")
print(now)

# Compare first and second timestamps
if df['tweet_date'][0] <= df['tweet_date'][1]:
    print("First entry is old")
else:
    print("Second entry is old")

2022-04-25 16:10:15.455446
2022-04-25 20:10:15.456443
First entry is old


In [4]:
# Compare newest tweet 
if pd.to_datetime(max(df['tweet_date'])).tz_convert(None) <= pd.to_datetime("today"):
    print("We only have data until yesterday")
else:
    print("YES, we can play now !")
    
print("Max tweet_date is: ",max(df["tweet_date"]))
print("Today is: ", pd.to_datetime("now"))

We only have data until yesterday
Max tweet_date is:  2022-04-24 02:27:31+00:00
Today is:  2022-04-25 20:10:16.364932


In [5]:
print(df.count())
#print(df.value_counts(normalize=True))
print(max(df["wordle_id"]))

wordle_id         474655
tweet_id          474655
tweet_date        474655
tweet_username    474655
tweet_text        474655
dtype: int64
308


In [6]:
if df.count == 204662:
    print("Nothing changed" )
else:
        # Hide today's answer
        print("For today's Wordle, my result was: ") #, (df.count))
        #print(df.count)    

if max(df["wordle_id"])<=246:
    print("For today´s running, my max wordle_id is: ", max(df["wordle_id"]) )

For today's Wordle, my result was: 


In [7]:
from collections import Counter
from itertools import product
import numpy as np
import pandas as pd
import pickle
import re
from scipy import spatial

In [8]:
vec_locs = sorted(["".join(x) for x in product("YMN", repeat=5)])
vec_locs[:5]

['MMMMM', 'MMMMN', 'MMMMY', 'MMMNM', 'MMMNN']

In [9]:
# Extract the Wordle result from a tweet, and remap the unicode feedback to Y/M/N
def wordle_guesses(tweet):
    text = (tweet.replace("Y", "y").replace("🟩", "Y")
                 .replace("M", "m").replace("🟨", "M")
                 .replace("N", "n").replace("⬛", "N").replace("⬜", "N"))
    guesses = re.findall("([YMN]+)", text)
    return guesses

def evaluate_guess_char(answer, guess, pos):
    if answer[pos]==guess[pos]:
        return "Y"
    unmatched_answer_chars = 0
    unmatched_guess_chars = 0
    this_guess_num = 0 
    for i in range(5):
        if answer[i]==guess[pos]:
            if answer[i]!=guess[i]:
                unmatched_answer_chars += 1
        if guess[i]==guess[pos]:
            if answer[i]!=guess[i]:
                unmatched_guess_chars += 1
                if i<pos:
                    this_guess_num += 1
    if this_guess_num<unmatched_answer_chars:
        return "M"
    return "N"

def evaluate_guess(answer, guess):
    return "".join(evaluate_guess_char(answer, guess, i) for i in range(5))

assert evaluate_guess("tools", "break")=="NNNNN"
assert evaluate_guess("tools", "tools")=="YYYYY"
assert evaluate_guess("tools", "tolls")=="YYNYY"
assert evaluate_guess("tools", "books")=="NYYNY"
assert evaluate_guess("tools", "broke")=="NNYNN"
assert evaluate_guess("tools", "yahoo")=="NNNMM"
assert evaluate_guess("tools", "bongo")=="NYNNM"
assert evaluate_guess("tools", "brook")=="NNYMN"
assert evaluate_guess("rates", "ranks")=="YYNNY"
assert evaluate_guess("rates", "apple")=="MNNNM"

In [10]:
with open("C:/Users/anjne/vec_all.pickle", "rb") as f:
    v_all = pickle.load(f)

with open("C:/Users/anjne/vec_ratio.pickle", "rb") as f:
    v_ratio = pickle.load(f)

with open("C:/Users/anjne/invalid_results.pickle", "rb") as f:
    invalid_results = pickle.load(f)
    
with open("C:/Users/anjne/answers.txt", "r") as f:
    answers = f.read().split("\n")

with open("C:/Users/anjne/other_words.txt", "r") as f:
    other_words = f.read().split("\n")

words = answers + other_words
answers.remove("agora") # NYTimes vesion seems to have skipped this

In [11]:
import pandas as pd
df = pd.read_csv("D:/study/Data Mining/Project/input/tweets.csv")
df

,wordle_id,tweet_id,tweet_date,tweet_username,tweet_text
0,210,1482553374591660037,2022-01-16 03:20:43+00:00,bpszebes,Wordle 210 4/6\n\n⬛⬛🟨🟨⬛\n🟩⬛⬛⬛⬛\n🟩🟩🟨⬛⬛\n🟩🟩🟩🟩🟩
1,210,1482553387937898499,2022-01-16 03:20:46+00:00,cruisecoup,Wordle 210 4/6\n\n⬜⬜⬜⬜⬜\n🟩🟩🟨🟨⬜\n🟩🟩⬜🟩🟨\n🟩🟩🟩🟩🟩
2,210,1482553422276698113,2022-01-16 03:20:55+00:00,DestroVega,Wordle 210 4/6\n\n⬜⬜⬜🟨⬜\n⬜🟩⬜🟨⬜\n⬜🟩⬜🟨⬜\n🟩🟩🟩🟩🟩
3,210,1482553436910628866,2022-01-16 03:20:58+00:00,brenmardash,Wordle 210 3/6\n\n⬜⬜🟨⬜⬜\n🟨🟨⬜⬜⬜\n🟩🟩🟩🟩🟩
4,210,1482553445726908420,2022-01-16 03:21:00+00:00,KatieHowse2,Wordle 210 3/6\n\n⬛⬛🟨⬛⬛\n🟩🟩🟩⬛⬛\n🟩🟩🟩🟩🟩
...,...,...,...,...,...
474650,308,1518053928977506304,2022-04-24 02:27:15+00:00,thisis_bza,Wordle 308 4/6\n\n⬛⬛⬛⬛🟩\n⬛⬛🟩⬛🟩\n⬛⬛🟩⬛🟩\n🟩🟩🟩🟩🟩
474651,308,1518053954424295424,2022-04-24 02:27:21+00:00,CherylWaite1,Wordle 308 4/6\n\n⬜⬜⬜🟨🟩\n🟨⬜⬜⬜🟩\n⬜⬜🟨⬜🟩\n🟩🟩🟩🟩🟩
474652,308,1518053961412100101,2022-04-24 02:27:23+00:00,SnortneyBell,Wordle 308 4/6\n\n⬛⬛⬛⬛⬛\n⬛⬛⬛🟨⬛\n⬛⬛🟨🟩🟩\n🟩🟩🟩🟩🟩
474653,308,1518053984325406720,2022-04-24 02:27:28+00:00,tylermaechelle,Wordle 308 5/6\n\n⬛🟩⬛🟨⬛\n⬛🟨⬛⬛⬛\n⬛🟩🟩⬛🟩\n⬛🟩🟩🟩🟩\n...


In [12]:
results = []

for i in sorted(set(df["wordle_id"])):
    tweet_texts = df[df["wordle_id"]==i]["tweet_text"]
    
    games = [wordle_guesses(tweet) for tweet in tweet_texts]   
    
    all_counts = Counter(res for game in games for res in game if len(game)>=2 and game[-1]=="YYYYY")
    first_counts = Counter(game[0] for game in games if len(game)>=2 and game[-1]=="YYYYY")
    penultimate_counts = Counter(game[-2] for game in games if len(game)>=2 and game[-1]=="YYYYY")
    
    vec_all = [all_counts[res] for res in vec_locs] 
    vec_first = [first_counts[res] for res in vec_locs] 
    vec_penultimate = [penultimate_counts[res] for res in vec_locs]
    vec_ratio = [penultimate_counts[res]/(all_counts[res]+1e-6) for res in vec_locs]
    
    dists = {"all": [], "ratio": []}
    
    for key in v_all:
        dists["all"].append((spatial.distance.cosine(vec_all, v_all[key]), key))
        dists["ratio"].append((spatial.distance.cosine(vec_ratio, v_ratio[key]), key))
    
    dists["invalid"] = [(len([g for g in games if any(r in invalid_results[key] for r in g)]), key) for key in v_all]

    ranks = {}
    for d in dists:
        s = sorted(dists[d])
        ranks[d] = {s[i][1]: i for i in range(len(s))}
    
    overall = []
    for key in v_all:
        overall.append((sum(ranks[r][key] for r in ranks), key))
    
    my_guess = sorted(overall)[0][1]
    answer = answers[i]
    answer_rank = [x[1] for x in sorted(overall)].index(answer)
        
    results.append([i, my_guess, answer, answer_rank, overall, dists, ranks])
    feedback = evaluate_guess(answer, my_guess).replace("Y", "🟩").replace("M", "🟨").replace("N", "⬛")

    if answer==my_guess:
        feedback += " !!!!!!"
    else:
        feedback += " :(:( the actual answer ranked %d on my guess list" % (answer_rank)
    
    if i<max(df["wordle_id"]):
        print("For Wordle %d, my guess was %s. %s" % (i, my_guess, feedback))
    else:
        # Hide today's answer
        print("For today's Wordle %d, my result was: %s" % (i, feedback))

For Wordle 210, my guess was panic. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 211, my guess was solar. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 212, my guess was shire. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 213, my guess was proxy. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 214, my guess was point. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 215, my guess was robot. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 216, my guess was prick. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 217, my guess was wince. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 218, my guess was crimp. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 219, my guess was knoll. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 220, my guess was sugar. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 221, my guess was whack. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 222, my guess was mount. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 223, my guess was cards. ⬛⬛🟩⬛⬛ :(:( the actual answer ranked 2 on my guess list
For Wordle 224, my guess was could. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 225, my guess was wrung. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 226, my guess was light. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 227, my guess was those. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 228, my guess was moist. 🟩🟩🟩🟩🟩 !!!!!!
For Wordle 229, my guess wa

In [13]:
# Unhide the output of this cell to reveal today's guess
print("For Wordle %d, my guess was %s. %s" % (i, my_guess, feedback))

For Wordle 308, my guess was olive. ⬛🟩🟨⬛⬛ :(:( the actual answer ranked 1499 on my guess list
